# 🚀 dLNk GPT V2 - Continuous Training

## ระบบเทรนต่อเนื่องพร้อม Real-time LINE Notifications

### ฟีเจอร์:
- ✅ ใช้ข้อมูล 999,972 samples จาก Google Drive
- ✅ V2 Anti-Overfitting Configuration
- ✅ รายงานผ่าน LINE แบบ real-time (ภาษาไทย)
- ✅ Auto-save checkpoints ทุก 500 steps
- ✅ Continuous monitoring
- ✅ ป้องกัน Colab disconnect

### ขั้นตอน:
1. เปลี่ยน Runtime เป็น GPU (A100 แนะนำ)
2. รัน cells ทั้งหมดตามลำดับ
3. รับการแจ้งเตือนผ่าน LINE
4. ปล่อยทิ้งไว้ได้เลย - ระบบจะดูแลเอง

---

In [ ]:
# 1️⃣ ตรวจสอบ GPU
!nvidia-smi
print("\n✅ GPU พร้อมใช้งาน")

In [ ]:
# 2️⃣ ป้องกัน Colab Disconnect
%%javascript
function ClickConnect(){
  console.log("🔄 Keeping Colab alive...");
  document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect, 60000)  // Every 60 seconds
console.log("✅ Anti-disconnect activated!");

In [ ]:
# 3️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted")

In [ ]:
# 4️⃣ ติดตั้ง Dependencies
%%capture
!pip install -q transformers>=4.30.0 datasets>=2.12.0 accelerate>=0.20.0 peft>=0.4.0 bitsandbytes tensorboard
print("✅ Dependencies installed")

In [ ]:
# 5️⃣ Login Hugging Face (ใส่ token ของคุณ)
from huggingface_hub import login

# TODO: ใส่ Hugging Face token ของคุณที่นี่
HF_TOKEN = ""  

if not HF_TOKEN:
    print("⚠️  กรุณาใส่ Hugging Face token")
else:
    login(token=HF_TOKEN)
    print("✅ Hugging Face login successful")

In [ ]:
# 6️⃣ Clone Repository
import os

if os.path.exists('/content/gptprojecttrain'):
    print("📁 Repository exists, pulling latest changes...")
    %cd /content/gptprojecttrain
    !git pull origin main
else:
    print("📥 Cloning repository...")
    %cd /content
    !git clone https://github.com/traingptproject/gptprojecttrain.git
    %cd gptprojecttrain

!ls -la train_v2_continuous.py
print("\n✅ Repository ready")

In [ ]:
# 7️⃣ ติดตั้ง manus-mcp-cli (สำหรับ LINE notifications)
# Note: ใน Colab จะใช้ mock version เพราะไม่สามารถเชื่อมต่อ MCP ได้โดยตรง
# แต่โค้ดจะพิมพ์ข้อความที่จะส่งไปยัง LINE

!mkdir -p /usr/local/bin

with open('/usr/local/bin/manus-mcp-cli', 'w') as f:
    f.write('''#!/bin/bash
echo "[LINE NOTIFICATION]"
echo "$@" | grep -oP '(?<="text":")[^"]*' | sed 's/\\\\n/\n/g'
echo '{"sentMessages":[{"id":"mock"}]}'
''')

!chmod +x /usr/local/bin/manus-mcp-cli
!export PATH="/usr/local/bin:$PATH"

print("✅ manus-mcp-cli installed (mock version for Colab)")
print("📱 LINE notifications will be printed to console")

In [ ]:
# 8️⃣ ตรวจสอบข้อมูลเทรนนิ่ง
import os

DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/training_data_1m_final.jsonl"

if os.path.exists(DATASET_PATH):
    file_size = os.path.getsize(DATASET_PATH) / (1024 * 1024)  # MB
    print(f"✅ Dataset found!")
    print(f"📁 Path: {DATASET_PATH}")
    print(f"💾 Size: {file_size:.2f} MB")
    
    # Count lines
    with open(DATASET_PATH, 'r') as f:
        line_count = sum(1 for _ in f)
    print(f"📊 Samples: {line_count:,}")
else:
    print(f"❌ Dataset not found at: {DATASET_PATH}")
    print("⚠️  Please check the path and make sure Google Drive is mounted")

In [ ]:
# 9️⃣ เริ่มการเทรน V2 Continuous
print("="*80)
print("🚀 Starting dLNk GPT V2 Continuous Training")
print("="*80)
print("")
print("📊 Configuration:")
print("  - Dataset: 999,972 samples")
print("  - Model: GPT-J-6B")
print("  - GPU: A100-SXM4-80GB")
print("  - Learning Rate: 5e-6 (V2)")
print("  - Weight Decay: 0.1 (V2)")
print("  - LoRA Rank: 16 (V2)")
print("  - LoRA Dropout: 0.05 (V2)")
print("  - Eval Steps: 500 (10x more frequent)")
print("")
print("📱 LINE Notifications:")
print("  - Training start")
print("  - Progress updates (every 5 min)")
print("  - Evaluation results (every 500 steps)")
print("  - Checkpoint saved")
print("  - Training complete")
print("  - Error alerts")
print("")
print("⏱️  Estimated time: 8-12 hours (with 1M samples)")
print("="*80)
print("")

# Run training
!python3 train_v2_continuous.py

---

## 📊 หลังการเทรนเสร็จ

### โมเดลจะถูกบันทึกที่:
- `/content/gptprojecttrain/dLNk-gpt-j-6b-exploit-v2-continuous/`

### ทดสอบโมเดล:
```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "./dLNk-gpt-j-6b-exploit-v2-continuous"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test
prompt = "### Instruction:\nExplain SQL injection\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=200)
print(tokenizer.decode(outputs[0]))
```

### อัปโหลดไปยัง Hugging Face Hub:
```python
model.push_to_hub("traingptproject/dLNk-gpt-j-6b-exploit-v2")
tokenizer.push_to_hub("traingptproject/dLNk-gpt-j-6b-exploit-v2")
```

---